# OrientDB tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [OrientDB SQL reference](http://www.orientdb.com/docs/last/SQL-Functions.html)
* [Orientdb python client](http://orientdb.com/docs/last/PyOrient-Client.html#working-with-the-client)

## Import libraries

In [25]:
import pyorient

In [26]:
ROOT_PASSWORD = "root"
client = pyorient.OrientDB("localhost", 2424)
session_id = client.connect("root", ROOT_PASSWORD)

In [27]:
print(client.db_list())

{{'databases': {}}}


## I. Quick start

### Creating the database

**Q:** Create a database `gods` as a `GRAPH_DATABASE` in `MEMORY_STORAGE_TYPE`. 

We will use it to store relationships between Greek deities.

In [28]:
# Creation d'une base de donnees
client.db_create("gods", pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY)

**Q:** Connect your pyorient client to the `gods` database.

In [29]:
# Ouverture de la base de donnees
client.db_open("gods", "root", ROOT_PASSWORD)

**Q:** You should now be able to launch OrientDB queries through the Python client with the [command()](http://orientdb.com/docs/last/PyOrient-Client-Command.html) function. 

You should think of OrientDB as a Graph-Document database for the following questions. Each vertex and edge will contain information on it inside a JSON document.

Create a new Vertex with content `{name: 'Zeus', symbol: 'thunder'}`. The [CREATE VERTEX : Create a vertex using JSON content](http://orientdb.com/docs/last/SQL-Create-Vertex.html) doc page should help you.

In [30]:
# Creation d'un vertex
client.command("CREATE VERTEX V CONTENT {'name': 'Zeus', 'symbol': 'thunder'}")

You have created a VERTEX in the previous question. The VERTEX is a [class](https://orientdb.com/docs/last/Tutorial-Classes.html) of OrientDB which defines a record that can be linked to others through EDGE instances.

You can find all VERTEX created in the database with a SQL command on the `V` table, like `SELECT * FROM V`. 

**Q:** Print all current vertices in `gods`, it should only have `Zeus` though for now.

In [31]:
# Définition de la requête
request = client.command("SELECT * FROM V")

# Affichage du résultat
for vertex in request:
    print(vertex.oRecordData)

{'name': 'Zeus', 'symbol': 'thunder'}


**Q:** Create new vertices with content : 
```
{name:Héra, symbol:tiara}
{name:Poséidon, symbol:trident}
{name:Athena, symbol:helmet}
{name:Arès, symbol:weapons} 
```

In [32]:
# Creation de plusieurs vertices
client.command("CREATE VERTEX V CONTENT {'name': 'Héra', 'symbol': 'tiara'}")
client.command("CREATE VERTEX V CONTENT {'name': 'Poséidon', 'symbol': 'trident'}")
client.command("CREATE VERTEX V CONTENT {'name': 'Athena', 'symbol': 'helmet'}")
client.command("CREATE VERTEX V CONTENT {'name': 'Arès', 'symbol': 'weapons'}")

**Q:** Display all vertices with name = `Arès`

In [33]:
# Définition de la requête
request = client.command("SELECT FROM V WHERE name = 'Arès'")

# Affichage du résultat
for vertex in request:
    print(vertex.oRecordData)

{'name': 'Arès', 'symbol': 'weapons'}


**Q:** Create an EDGE from `Zeus` to `Poséidon` with the content `{kind: 'sibling'}

In [34]:
# Création d'une relation entre deux vertices
client.command("CREATE EDGE E FROM (SELECT FROM V WHERE name = 'Zeus') TO (SELECT FROM V WHERE name = 'Poséidon') CONTENT {'kind': 'sibling'}")

**Q:** Redisplay all vertices, discuss.

In [35]:
# Définition de la requête
request = client.command("SELECT * FROM V")

# Affichage du résultat
for vertex in request:
    print(vertex.oRecordData)

{'name': 'Zeus', 'symbol': 'thunder', 'out_': <pyorient.otypes.OrientBinaryObject object at 0x0000029787357EB8>}
{'name': 'Héra', 'symbol': 'tiara'}
{'name': 'Poséidon', 'symbol': 'trident', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x000002978740E198>}
{'name': 'Athena', 'symbol': 'helmet'}
{'name': 'Arès', 'symbol': 'weapons'}


**Q:** Display all edges. They are contained in the class `E`

In [36]:
# Définition de la requête
request = client.command("SELECT * FROM E")

# Affichage du résultat
for edge in request:
    print(edge.oRecordData)

{'kind': 'sibling', 'out': <pyorient.otypes.OrientRecordLink object at 0x000002978740E4A8>, 'in': <pyorient.otypes.OrientRecordLink object at 0x000002978740E550>}


Two fields on vertices have appeared, containing the outgoing (out_) and incoming (in_) links.

At the edge level, two fields point to the original (out) and destination (in) vertices.

**Q:** Lets create some more edges :

* Zeus > Héra (sibling)
* Zeus > Arès (father)
* Zeus > Athena (father)
* Héra > Arès (mother)
* Héra > Zeus (sibling)
* Poséidon > Zeus (sibling)

_Hint 1 :_ check [the CREATE EDGE doc page](http://orientdb.com/docs/last/SQL-Create-Edge.html) to find an example for creating edges on vertices using subqueries so you can run queries to fetch the vertices before creating an edge in between.

_Hint 2 :_ after you have found the command to create edges between vertices with sub-queries, you should be well-versed enough in Python to create a list of all edges in the question, and loop the command on each element of the list to create all edges in one go =)

In [37]:
# Création de plusieurs relations
edges = [
    {'from': 'Zeus', 'to': 'Héra', 'kind': 'sibling'},
    {'from': 'Zeus', 'to': 'Arès', 'kind': 'father'},
    {'from': 'Zeus', 'to': 'Athena', 'kind': 'father'},
    {'from': 'Héra', 'to': 'Arès', 'kind': 'mother'},
    {'from': 'Héra', 'to': 'Zeus', 'kind': 'sibling'},
    {'from': 'Poséidon', 'to': 'Zeus', 'kind': 'sibling'}
]

# Application des relations
for edge in edges:
    client.command(f"CREATE EDGE E FROM (SELECT FROM V WHERE name = '{edge['from']}') TO (SELECT FROM V WHERE name = '{edge['to']}') CONTENT {{'kind': '{edge['kind']}'}}")

### Looking for data

**Q:** Using [out()](http://orientdb.com/docs/last/Tutorial-Working-with-graphs.html#querying-graphs) function, display all vertices connected and outgoing from Zeus.

You should use the EXPAND() special function to transform the vertex collection in the result-set by expanding it, making the results more readable.

In [38]:
# Définition de la requête
request = client.command("SELECT EXPAND(out()) FROM V WHERE name = 'Zeus'")

# Affichage du résultat
for vertex in request:
    print(vertex.oRecordData)

{'name': 'Poséidon', 'symbol': 'trident', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x000002978740E908>, 'out_': <pyorient.otypes.OrientBinaryObject object at 0x000002978740E828>}
{'out_': <pyorient.otypes.OrientBinaryObject object at 0x000002978740EA58>, 'name': 'Héra', 'symbol': 'tiara', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x000002978740EA20>}
{'in_': <pyorient.otypes.OrientBinaryObject object at 0x000002978740EC50>, 'name': 'Arès', 'symbol': 'weapons'}
{'name': 'Athena', 'symbol': 'helmet', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x000002978740ED30>}


**Q:** Display all vertices which got a father (the vertices which are the destination of an arc whose kind attribute is father).

_Hint: You can notice that we use the field `in` the arc, and not the function `in()` which applies to vertices._

In [39]:
# Définition de la requête
query = "SELECT FROM V WHERE @rid IN (SELECT in FROM E WHERE kind = 'father')"
request = client.command(query)

# Affichage du résultat
for vertex in request:
    print(vertex.oRecordData)

{'name': 'Athena', 'symbol': 'helmet', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x000002978740E438>}
{'in_': <pyorient.otypes.OrientBinaryObject object at 0x000002978740EEB8>, 'name': 'Arès', 'symbol': 'weapons'}


**Q:** As in SQL, the operator `in` used in a clause `where` allows to restrict the possible values with an embedded query _(where ... in (select ...))_. 

Display the mothers, by displaying the vertices where an outgoing arc is part of the arcs where kind is a mother.

In [40]:
# Définition de la requête
query = "SELECT FROM V WHERE @rid IN (SELECT out FROM E WHERE kind = 'mother')"
request = client.command(query)

# Affichage du résultat
for vertex in request:
    print(vertex.oRecordData)

{'out_': <pyorient.otypes.OrientBinaryObject object at 0x000002978740ECC0>, 'name': 'Héra', 'symbol': 'tiara', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x000002978740E940>}


**Q:** Display the brothers and sisters of Zeus (the destination summits of an arc whose kind is sibling and whose original summit is Zeus).

In [41]:
# Définition de la requête
query = "SELECT expand(in) from E where out.name like 'Zeus' and kind like 'sibling'"
request = client.command(query)

# Affichage du résultat
for vertex in request:
    print(vertex.oRecordData)

{'name': 'Poséidon', 'symbol': 'trident', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x000002978740E438>, 'out_': <pyorient.otypes.OrientBinaryObject object at 0x000002978740E4A8>}
{'name': 'Héra', 'out_': <pyorient.otypes.OrientBinaryObject object at 0x000002978740E390>, 'symbol': 'tiara', 'in_': <pyorient.otypes.OrientBinaryObject object at 0x000002978740E6D8>}


## Modeling a Product Recommendation System

You are currently modeling the data of a product recommendation system with OrientDB.

The main purpose of such a system is to answer the question "which products were purchased by their people who purchased product X? »

Purchased products have only one name field. They are purchased by people who have a nickname.

When a person buys a product, the date of purchase is stored. 

Instead of working with "anonymous" vertices and arcs, you will use classes. The `create class` command allows you to create custom classes.

The vertex classes must extend V, the arc classes must extend E.

**Q:** Create an `eCommerce` database, and the necessary classes to model the system.

PS : you can view all classes in the database with :

```python
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)
```

In [42]:
# Création de la base de données eCommerce
client.db_create("eCommerce", pyorient.DB_TYPE_GRAPH, pyorient.STORAGE_TYPE_MEMORY)

# Connexion à la base de données eCommerce
client.db_open("eCommerce", "root", ROOT_PASSWORD)

# Création des classes de vertex
client.command("CREATE CLASS Person EXTENDS V")
client.command("CREATE PROPERTY Person.nickname STRING")

client.command("CREATE CLASS Product EXTENDS V")
client.command("CREATE PROPERTY Product.name STRING")

# Création des classes d'arêtes
client.command("CREATE CLASS Purchase EXTENDS E")
client.command("CREATE PROPERTY Purchase.date DATE")

# Affichage des classes
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)

{{'name': 'Product'},'version':0,'rid':'#-2:0'}
{{'name': 'ORole'},'version':0,'rid':'#-2:1'}
{{'name': 'Person'},'version':0,'rid':'#-2:2'}
{{'name': 'OIdentity'},'version':0,'rid':'#-2:3'}
{{'name': 'E'},'version':0,'rid':'#-2:4'}
{{'name': 'V'},'version':0,'rid':'#-2:5'}
{{'name': 'ORestricted'},'version':0,'rid':'#-2:6'}
{{'name': 'OSchedule'},'version':0,'rid':'#-2:7'}
{{'name': 'OFunction'},'version':0,'rid':'#-2:8'}
{{'name': 'Purchase'},'version':0,'rid':'#-2:9'}
{{'name': 'OSequence'},'version':0,'rid':'#-2:10'}
{{'name': 'OUser'},'version':0,'rid':'#-2:11'}
{{'name': 'OTriggered'},'version':0,'rid':'#-2:12'}


**Q:** Create the following products: `spaghetti`, `bolognese sauce`, `cheese`, `apple`.

In [43]:
# Création des produits
client.command("CREATE VERTEX Product CONTENT {'name': 'spaghetti'}")
client.command("CREATE VERTEX Product CONTENT {'name': 'bolognese sauce'}")
client.command("CREATE VERTEX Product CONTENT {'name': 'cheese'}")
client.command("CREATE VERTEX Product CONTENT {'name': 'apple'}")

# Affichage des produits
for product in client.command("SELECT * FROM Product"):
    print(product.oRecordData)

{'name': 'spaghetti'}
{'name': 'bolognese sauce'}
{'name': 'cheese'}
{'name': 'apple'}


**Q:** Create the following people: `peter`, `meredith`.

In [44]:
# Création des personnes
client.command("CREATE VERTEX Person CONTENT {'nickname': 'peter'}")
client.command("CREATE VERTEX Person CONTENT {'nickname': 'meredith'}")

# Affichage des personnes
for person in client.command("SELECT * FROM Person"):
    print(person.oRecordData)

{'nickname': 'peter'}
{'nickname': 'meredith'}


**Q:** Create the following purchases: 
- peter > spaghetti + cheese on 20/01/2016 
- meredith > cheese + apple + bolognese sauce on 22/01/2016
- peter > spaghetti + bolognese sauce on 27/01/2016


In [45]:
# Création des achats
purchases = [
    {'person': 'peter', 'products': ['spaghetti', 'cheese'], 'date': '2016-01-20'},
    {'person': 'meredith', 'products': ['cheese', 'apple', 'bolognese sauce'], 'date': '2016-01-22'},
    {'person': 'peter', 'products': ['spaghetti', 'bolognese sauce'], 'date': '2016-01-27'}
]

# Application des achats
for purchase in purchases:
    person = purchase['person']
    products = purchase['products']
    date = purchase['date']
    for product in products:
        client.command(f"CREATE EDGE Purchase FROM (SELECT FROM Person WHERE nickname = '{person}') TO (SELECT FROM Product WHERE name = '{product}') CONTENT {{'date': '{date}'}}")

# Affichage des achats
for purchase in client.command("SELECT * FROM Purchase"):
    print(purchase.oRecordData)

{'date': datetime.date(2016, 1, 20), 'out': <pyorient.otypes.OrientRecordLink object at 0x000002978734EB00>, 'in': <pyorient.otypes.OrientRecordLink object at 0x000002978741BD30>}
{'date': datetime.date(2016, 1, 20), 'out': <pyorient.otypes.OrientRecordLink object at 0x000002978741BEB8>, 'in': <pyorient.otypes.OrientRecordLink object at 0x000002978741BF98>}
{'date': datetime.date(2016, 1, 22), 'out': <pyorient.otypes.OrientRecordLink object at 0x000002978741D160>, 'in': <pyorient.otypes.OrientRecordLink object at 0x000002978741D240>}
{'date': datetime.date(2016, 1, 22), 'out': <pyorient.otypes.OrientRecordLink object at 0x000002978741D3C8>, 'in': <pyorient.otypes.OrientRecordLink object at 0x000002978741D4A8>}
{'date': datetime.date(2016, 1, 22), 'out': <pyorient.otypes.OrientRecordLink object at 0x000002978741D630>, 'in': <pyorient.otypes.OrientRecordLink object at 0x000002978741D710>}
{'date': datetime.date(2016, 1, 27), 'out': <pyorient.otypes.OrientRecordLink object at 0x0000029787

**Q:** Who bought Bolognese sauce?

In [46]:
# Définition de la requête
query = "SELECT FROM Person WHERE @rid IN (SELECT out FROM Purchase WHERE in = (SELECT FROM Product WHERE name = 'bolognese sauce'))"
request = client.command(query)

# Affichage du résultat
for person in request:
    print(person.oRecordData)

**Q:** It is possible to link the `out` and `in` navigation functions. What products are purchased with Bolognese sauce? 

In [ ]:
# Définition de la requête
query = """
SELECT DISTINCT(name) 
FROM (
  SELECT expand(out('Purchase')) AS product
  FROM Person
  WHERE @rid IN (
    SELECT expand(in('Purchase')) 
    FROM Product 
    WHERE name = 'bolognese sauce'
  )
)
WHERE name <> 'bolognese sauce'
"""
results = client.command(query)

# Affichage du résultat
for product in results:
    print(product.oRecordData)

{'DISTINCT': 'spaghetti'}
{'DISTINCT': 'cheese'}
{'DISTINCT': 'apple'}


## Postquisites

Since we create databases in memory, they get destroyed on server shutdown.